In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from featuresv2 import features, targets, metadata
%matplotlib inline

## Version 2 Model

In [4]:
data = pd.read_csv('outputs/model_training_data_v2.csv')

In [5]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [6]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [7]:
targets

['homeTeamScore.matchScore.totalScore',
 'awayTeamScore.matchScore.totalScore',
 'score_diff']

In [8]:
def generate_test_train_split(data, features, target, test_size=0.3):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [9]:
def tpot_pipeline_v2(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], data_['y_train'])
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [10]:
def return_stats_inputv2(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
#     if 'team.name' not in model_inputs:
#         model_inputs.append('team.name')
        
    match_ups = data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)]
    stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').mean()
    
    return stats[model_inputs]

In [11]:
def generate_margin_tear_offv2(model, round_matchups, team_stats, model_inputs):
    print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        predict = model.predict(return_stats_inputv2(team_stats, tup[0], tup[1], model_inputs))[0]
        if predict < 0:
            winner = tup[1]
        else:
            winner = tup[0]
        print("------------------------------------------------")
        print("Home: " + tup[0])
        print("Away: " + tup[1])
        print("Winner: " + winner + "\t       " + "Margin: " + str(abs(round(predict))))

In [44]:
def generate_score_tear_offv2(home_model, away_model, round_matchups, team_stats, model_inputs):
   # print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        h_predict = home_model.predict(return_stats_inputv2(team_stats, tup[0], tup[1], model_inputs))[0]
        a_predict = away_model.predict(return_stats_inputv2(team_stats, tup[0], tup[1], model_inputs))[0]
        if h_predict < a_predict:
            winner = tup[1]
            margin = round(a_predict) - round(h_predict)
        else:
            winner = tup[0]
            margin = round(h_predict) - round(a_predict)
        paddingh = 25 - len("Home: " + tup[0])
        paddinga = 25 - len("Away: " + tup[1])
        print("------------------------------------------------")
        print("Home: " + tup[0] + " "*paddingh + str(abs(round(h_predict))))
        print("Away: " + tup[1] + " "*paddinga + str(abs(round(a_predict))))
        print("Winner: " + winner + " by "+ str(margin) + " points")
        print("Total Score: " + str(round(h_predict) + round(a_predict)))

In [13]:
home_model = tpot_pipeline_v2(data, features, targets[0], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9110022771491838

Generation 2 - Current best internal CV score: 0.9110022771491838

Generation 3 - Current best internal CV score: 0.9715273338099124

Generation 4 - Current best internal CV score: 0.9715273338099124

Generation 5 - Current best internal CV score: 0.9715273338099124

Best pipeline: RidgeCV(input_matrix)
0.9733412515746034


In [14]:
away_model = tpot_pipeline_v2(data, features, targets[1], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9724996542190854

Generation 2 - Current best internal CV score: 0.9725172607697754

Generation 3 - Current best internal CV score: 0.9725172607697754

Generation 4 - Current best internal CV score: 0.9735639273090172

Generation 5 - Current best internal CV score: 0.9735639273090172

Best pipeline: ElasticNetCV(RobustScaler(input_matrix), l1_ratio=1.0, tol=0.001)
0.9725013754589164


In [15]:
margin_model = tpot_pipeline_v2(data, features, targets[2], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9731446552616015

Generation 2 - Current best internal CV score: 0.9735916352881799

Generation 3 - Current best internal CV score: 0.9735916352881799

Generation 4 - Current best internal CV score: 0.9735916352881799

Generation 5 - Current best internal CV score: 0.9735916352881799

Best pipeline: ElasticNetCV(MinMaxScaler(input_matrix), l1_ratio=1.0, tol=0.0001)
0.9720474035364391


In [46]:
generate_score_tear_offv2(home_model, away_model, round1, data, features)

------------------------------------------------
Home: Carlton            80
Away: Richmond           83
Winner: Richmond by 3 points
Total Score: 163
------------------------------------------------
Home: Collingwood        61
Away: Sydney Swans       47
Winner: Collingwood by 14 points
Total Score: 108
------------------------------------------------
Home: Essendon           98
Away: Hawthorn           81
Winner: Essendon by 17 points
Total Score: 179
------------------------------------------------
Home: GWS Giants         57
Away: North Melbourne    84
Winner: North Melbourne by 27 points
Total Score: 141
------------------------------------------------
Home: Geelong Cats       94
Away: St Kilda           67
Winner: Geelong Cats by 27 points
Total Score: 161
------------------------------------------------
Home: Gold Coast Suns    89
Away: Adelaide Crows     86
Winner: Gold Coast Suns by 3 points
Total Score: 175
------------------------------------------------
Home: Melbourne     

In [26]:
generate_margin_tear_offv2(margin_model, round1, data, features)

-------------------MinMaxScaler TEAR-OFF-------------------
------------------------------------------------
Home: Carlton
Away: Richmond
Winner: Richmond	       Margin: 3
------------------------------------------------
Home: Collingwood
Away: Sydney Swans
Winner: Collingwood	       Margin: 15
------------------------------------------------
Home: Essendon
Away: Hawthorn
Winner: Essendon	       Margin: 17
------------------------------------------------
Home: GWS Giants
Away: North Melbourne
Winner: North Melbourne	       Margin: 27
------------------------------------------------
Home: Geelong Cats
Away: St Kilda
Winner: Geelong Cats	       Margin: 27
------------------------------------------------
Home: Gold Coast Suns
Away: Adelaide Crows
Winner: Gold Coast Suns	       Margin: 2
------------------------------------------------
Home: Melbourne
Away: Western Bulldogs
Winner: Melbourne	       Margin: 29
------------------------------------------------
Home: Port Adelaide
Away: West C